# Sending Data to FastAPI


## Motivation

For many types of API query, it is necessary to send data to the API along with your request. Here are some examples of why you might need to send data to an API:

- Make a GET request for some filtered subset of the data provided by the API (eg. listing all products in a specific category)
- Make POST requests where you supply data to the API, for example when making a post on a social media website
- Authenticating yourself to prove that you are able to access a specific service
- Sending an image to the API of a machine learning model

## Sending Data via the Query String

You can send individual parameters to your API via the URL query string. As previously discussed, the query parameters come after the endpoint URL, separated from it with a `?` symbol, and take the form of key-value pairs separated by ampersands (`&`):

```
my_example_api.com/this_query?parameter1=value1&parameter2=value2
```

The associated endpoint function then looks for matches between the supplied parameter keys and the named parameters in the function definition:

In [ ]:
import fastapi
import uvicorn

api = fastapi.FastAPI()

@api.get('/book/by_genre')
def get_book_by_genre(genre: str):
    # This function is looking for a query parameter called 'genre'
    print(f'Getting books in genre: {genre}')


Sending a query to this endpoint such as the following:

In [ ]:
import requests
response = requests.get("http://127.0.0.1:8000/book/by_genre?genre=scifi")

Will assign the string value `scifi` to the parameter `genre` in the function associated with the api decorator. The response will look like this:

<div style="text-align:center"><img src="images/booksingenre.gif" width=500/></div>

## Sending Data via the Path:

Another way to send values in an API request is via the URI path itself. You can define a variable inside braces `{}` in the URI path referenced in the API decorator, and if this matches the name of a parameter in the associated function, then the value will that appears at that part of the query will be passed to the function as an argument. 

For example, the function `get_book_by_ID` defined below is looking for an integer input parameter called `book_id`, and the location of the parameter in the URI has been defined by the location of `{book_id}`.

In [ ]:
import fastapi
import uvicorn

api = fastapi.FastAPI()

@api.get('/book/by_id/{book_id}')
def get_book_by_ID(book_id: int):
    # This function is looking for a query parameter called 'genre'
    print(f'Fetching book with id: {book_id}')

We can then send a book ID in the URI query as follows:

In [ ]:
import requests
response = requests.get("http://127.0.0.1:8000/book/by_id/1234")

and the response will look like this:

<div style="text-align:center"><img src="images/book_by_id.gif" width=500/></div>



## Sending Data in the Request Body

The third way to send data to an API is in the request body. The format here is to pass the endpoint URI, along with some value associated with the `data` keyword argument.

Lets imagine we wanted to add a feature to our book shop API where we can add a new author to our database. We want to create a POST request endpoint and have it parse in a dictionary of key-value pairs representing the information fields we want to add for a given author.

To send this information to the endpoint is easy. We just need to create a dictionary of the data we wish to add, and then serialise it into JSON format using the `json.dumps()` method, and pass it as the value associated with the `data` keyword argument in the request.

In [ ]:
import requests
import json

author_data = {'name': 'Isaac Asimov', 'genre': 'science fiction', 'author_id': 1234}

response = requests.post("http://127.0.0.1:8000/author/new_author/", data=json.dumps(author_data))

But how do we set up the API to interpret our input data here? By default, FastAPI will look for the input parameters to the function in the query string. To tell it to look for it in the request body instead, we need to define a class, and then if it finds a match for the specification of that class, it will use the data in the body rather than the data in the query string.

## Introducing the Pydantic Model

> Pydantic is a data validation library in Python that provides a way to validate that the incoming data structures are as expected. It enables you to define what data your application can receive, and Pydantic will ensure that incoming data corresponds to your model, taking care of validation and error handling.

Pydantic models are built around Pydantic's `BaseModel` class. To build your own datamodel, you can create a class that inherits from `BaseModel` and then specify your data structure within it. For example, we could create an `Author` class as follows:

In [ ]:
from pydantic import BaseModel

class Author(BaseModel):
    name: str
    genre: str = None
    author_id: int
    

We can now refer to this class in the function associated with our `.post(/author/new_author)` endpoint:

In [ ]:
import fastapi
import uvicorn

api = fastapi.FastAPI()

@api.post('/author/new_author')
def create_new_author(author: Author):
   print(f"adding new author, name: {author.name}, id: {author.author_id}")

FastAPI will look for some data that fits this model in the body of the request, and if it finds a match, it will pass the variable to the input of the function.

<div style="text-align:center"><img src="images/body.gif" width=800/></div>